In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
import statsmodels.api as sm
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import  train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
from xgboost import plot_importance
from sklearn.inspection import permutation_importance

In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna import Trial

optuna.__version__

In [3]:
#Import LGBM
import lightgbm as lgb
from xgboost import XGBRegressor
import xgboost as xgb

## Config

In [5]:
# Matplotlib Config
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Pandas and numpy config
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Helper Functions

In [6]:
def generate_metrics(Y_test,Y_predicted):
    mse = mean_squared_error(Y_test, Y_predicted)
    rmse = math.sqrt(mean_squared_error(Y_test, Y_predicted))
    mae = mean_absolute_error(Y_test, Y_predicted)
    rsquare_score = r2_score(Y_test, Y_predicted)
    return round(mse,2), round(rmse,2), round(mae,2) , round(rsquare_score,2)

In [7]:
def missing_values_table(input_df):
    """
    Returns the number of missing values in each column (if it has any missing values) and percentage of missing values.

    Parameters
    ----------
    input_df: pd.DataFrame
        The dataframe that whose missing data information is required 

    Returns
    -------
    mis_val_table_ren_columns: pd.DataFrame
        Returns a dataframe containing columns and missing data information

    """
    # Total missing values
    mis_val = input_df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * input_df.isnull().sum() / len(input_df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Values Missing'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Values Missing', ascending=False).round(1)

    # Print some summary information
    print ("Your selected dataframe has " + str(input_df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns

## Load Data

In [8]:
pip install gdown

In [9]:
!gdown https://drive.google.com/uc?id=1NGPdXsplAbbGf1Na7Kn3x9netwZXVyGw
!gdown https://drive.google.com/uc?id=1M_qVhIVMqgsCs5OS-bGCxIZB8jTPaeqr

In [10]:
raw_data = pd.concat([pd.read_csv('./sao_paulo_2019_v6.csv'), pd.read_csv('./london_2019_v3.csv')]).reset_index(drop = True)

In [11]:
raw_data.head(5)

In [12]:
raw_data = raw_data.drop(['Unnamed: 0'], axis = 1)

In [13]:
missing_values_table(raw_data)

## Exploratory Data Analysis

In [14]:
raw_data.osm_way_id.nunique()

## Data Preparation

In [15]:
raw_data.columns

In [16]:
raw_data.speed_mph_mean.describe()

In [17]:
sao_paulo_pop_mean= raw_data[raw_data.Country_Name == 'Brazil']['population_density_start'].mean()

In [18]:
raw_data['hospital_address_start'] = raw_data['hospital_address_start'].fillna(0)
raw_data['hospital_address_end'] = raw_data['hospital_address_end'].fillna(0)
raw_data['School_Name_end'] = raw_data['School_Name_end'].fillna(0)
raw_data['School_Name_start'] = raw_data['School_Name_start'].fillna(0)
raw_data['start_area_code'] = raw_data['start_area_code'].fillna('missing')
raw_data['area_names_start'] = raw_data['area_names_start'].fillna('missing')
raw_data['end_area_code'] = raw_data['end_area_code'].fillna('missing')
raw_data['area_names_end'] = raw_data['area_names_end'].fillna('missing')

raw_data['population_density_start'] = raw_data['population_density_start'].fillna(sao_paulo_pop_mean)
raw_data['population_density_end'] = raw_data['population_density_end'].fillna(sao_paulo_pop_mean)

In [19]:
missing_values_table(raw_data)

In [20]:
## Column definitions
numerical_columns = ['month', 'day', 'hour', 'start_lat', 'start_lon', 'end_lat', 'end_lon',\
                   'TEMP_in_C', 'VISIB_in_miles', 'Total_precipitation_in_inches', 'Fog', 'Rain',\
                   'Snow', 'Hail', 'Thunder', 'Tornado', 'Quality_of_roads', 'Road_connectivity_index', 'Roads','population_density_start',\
                   'population_density_end', 'School_Name_start', 'School_Name_end', 'hospital_address_start', 'hospital_address_end']
  
category_columns = ['start_area_code', 'end_area_code', 'area_names_end', 'area_names_start', 'Country_Name']



target_column = 'speed_mph_mean'

In [21]:
filtered_data = raw_data[numerical_columns + category_columns + [target_column]].copy()

In [22]:
filtered_data.head()

In [23]:
sns.histplot(x = 'speed_mph_mean', data = filtered_data, )

In [24]:
one_hot_columns = category_columns
for col in one_hot_columns:
    filtered_data[col] = filtered_data[col].astype('category')

## Split Test Train

In [25]:
'''Generating a 80%-20% split between train/test datasets'''

X = filtered_data.loc[:, filtered_data.columns != target_column]
Y = filtered_data.loc[:, filtered_data.columns == target_column]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2) 

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

## Modeling

## XGB

In [49]:
one_hot_pipeline = ColumnTransformer([('categorical', OneHotEncoder(handle_unknown='ignore'), one_hot_columns)], remainder='passthrough')
# one_hot_pipeline = ColumnTransformer([('categorical', LabelEncoder(), one_hot_columns)], remainder='passthrough')
encoder = one_hot_pipeline.fit(X_train)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)

# X_train[one_hot_columns] =  X_train[one_hot_columns].apply(LabelEncoder().fit_transform)
# X_test[one_hot_columns] =  X_test[one_hot_columns].apply(LabelEncoder().fit_transform)

In [50]:
params = {
    'n_estimators':15, 
    'max_depth':20,
    
}
XG_base = XGBRegressor(**params, verbosity=0)
baseline_model_xg = XG_base.fit(X_train, y_train)

In [51]:
baseline_test_predictions = baseline_model_xg.predict(X_test)

In [52]:
mse, rmse, mae, rsquare_score = generate_metrics(y_test,baseline_test_predictions)

In [53]:
print("Mean square error is:{}".format(mse))
print("Root mean square error is :{}".format(rmse))
print("Mean absolute error is :{}".format(mae))
print("R Squared Score is :{}".format(rsquare_score))

In [54]:
# plot_importance(XG_base)
# plt.show()

## Hyperparameter tuning using optuna

In [55]:
def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
#         'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators', 20, 400),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': 24,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(X_test)
    
    rmse = mean_squared_error(y_test, preds,squared=False)
    
    return rmse

In [56]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [57]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [58]:
# After 100 trials
optimal_params ={'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.6,
 'enable_categorical': False,
 'gamma': 0,
 'gpu_id': -1,
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 17,
 'min_child_weight': 3,
 'missing': np.nan,
 'monotone_constraints': '()',
 'n_estimators': 338,
 'n_jobs': 2,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 0.685789526,
 'reg_lambda': 0.335092366,
 'scale_pos_weight': 1,
 'subsample': 1.0,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': 0,
 'lambda': 0.33509236026309625,
 'alpha': 0.6857894969087497}

In [67]:
XG_base2 = XGBRegressor(**optimal_params)
baseline_model_xg2 = XG_base2.fit(X_train, y_train)

In [68]:
baseline_test_predictions2 = baseline_model_xg2.predict(X_test)

In [69]:
mse, rmse, mae, rsquare_score = generate_metrics(y_test,baseline_test_predictions2)
print("Mean square error is:{}".format(mse))
print("Root mean square error is :{}".format(rmse))
print("Mean absolute error is :{}".format(mae))
print("R Squared Score is :{}".format(rsquare_score))

In [86]:
baseline_train_predictions2 = baseline_model_xg2.predict(X_train)

In [87]:
mse, rmse, mae, rsquare_score = generate_metrics(y_train,baseline_train_predictions2)
print("Mean square error is:{}".format(mse))
print("Root mean square error is :{}".format(rmse))
print("Mean absolute error is :{}".format(mae))
print("R Squared Score is :{}".format(rsquare_score))

In [70]:
baseline_model_xg2.get_params()

## Baseline Model

In [82]:
baseline_mse, baseline_rmse, baseline_mae, baseline_rsquare_score = generate_metrics(y_test,len(y_test)*[filtered_data.speed_mph_mean.mean()])

In [83]:
print("Mean square error is:{}".format(baseline_mse))
print("Root mean square error is :{}".format(baseline_rmse))
print("Mean absolute error is :{}".format(baseline_mae))
print("R Squared Score is :{}".format(baseline_rsquare_score))

In [84]:
baseline_mse, baseline_rmse, baseline_mae, baseline_rsquare_score = generate_metrics(y_train,len(y_train)*[filtered_data.speed_mph_mean.mean()])

In [85]:
print("Mean square error is:{}".format(baseline_mse))
print("Root mean square error is :{}".format(baseline_rmse))
print("Mean absolute error is :{}".format(baseline_mae))
print("R Squared Score is :{}".format(baseline_rsquare_score))